In [1]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()

In [3]:
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['Price'] = housing.target

In [4]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Train, test, model hyperparamenter tuning and mlflow experiments

In [5]:
from urllib.parse import urlparse

###divide data

X = data.drop('Price', axis=1)
y = data['Price']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Hyperparamenter tuning

In [ ]:
def hyperparameter_tuning(x_train, y_train, param_grid):

    with mlflow.start_run():
        # Create a RandomForestRegressor
        rf = RandomForestRegressor() 
        # Perform grid search
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2,
                                   scoring='neg_mean_squared_error')
        grid_search.fit(x_train, y_train)

        print(grid_search.best_params_['n_estimators'])

        # Get the best hyperparameters
        best_params = grid_search.best_params_
        mlflow.log_params(best_params)

        return grid_search

In [ ]:
from mlflow.models import infer_signature

signature = infer_signature(x_train, y_train)

param_grid = {
    'n_estimators': [25, 55],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

### Ensure no previous run is active
if mlflow.active_run():
    mlflow.end_run()

### start MLFLOW experiment
with mlflow.start_run():
    ### hyperparameter tuning
    grid_search = hyperparameter_tuning(x_train, y_train, param_grid)

    ### Get the best hyperparameters
    best_model = grid_search.best_estimator_

    ### Evaluate the best model
    y_pred = best_model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)

    ### Log the best parameters and metrics
    mlflow.log_params(grid_search.best_params_) #log params expects a dictionary

    mlflow.log_metric("mse", mse)

    ### Tracking url

    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")  ## if I dont sent this, it will assign a "file" name to the tracking uri
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme ## Scheme: basically ip address

    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_model, "model", signature=signature, registered_model_name="Best rf model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)

    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
    print("Model best hyperparameter %s" % grid_search.best_params_)
    print("Model mse %s" % mse)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=25; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=25; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=25; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=25; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=55; total time=   2.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=55; total time=   2.3s
[CV] END max_depth=10, min_samples_leaf

2025/03/15 16:13:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best rf model'.
2025/03/15 16:13:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best rf model, version 1


Model saved in run be73e2f9ef724cd29ca22c4466999974
Model best hyperparameter {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 55}
Model mse 0.2565516384696525


Created version '1' of model 'Best rf model'.
